# Pong: Sintesi Tecnica e Concettuale

## 1. Setup Ambientale: Camera e Spazio Visivo

### Proiezione Ortografica (2D)
Nella proiezione ortografica, l'asse Z non influenza la dimensione degli oggetti. Non c'è prospettiva: gli oggetti mantengono la stessa dimensione indipendentemente dalla distanza.

### Risoluzione Fissa
Impostare una risoluzione fissa (640x360) garantisce che il gameplay sia bilanciato durante lo sviluppo.

- **Risoluzione**: Numero effettivo di pixel (640x360)
- **Aspect Ratio**: Rapporto proporzionale (16:9)

### Canvas Scaler
Il Canvas Scaler adatta l'interfaccia UI mantenendo le proporzioni:
- **UI Scale Mode**: Scale With Screen Size
- **Reference Resolution**: 640x360
- **Match**: Height (1.0) - per giochi landscape come Pong

---

## 2. Scaling dello Sfondo: Matematica dei Pixel

### Il Problema
Lo sfondo è alto 320 pixel, ma lo schermo è 360 pixel. Devono combaciare perfettamente.

### La Formula

$$\text{Scale Factor} = \frac{\text{Risoluzione Schermo}}{\text{Altezza Immagine Originale}}$$

$$\text{Scale Factor} = \frac{360}{320} = 1.125$$

### Applicazione
Impostare lo **Scale Y a 1.125** nel Transform dello sfondo farà sì che copra esattamente l'altezza del gioco.

---

## 3. Mapping Pixel-Unità: PPU (Pixels Per Unit)

### Il Concetto
Unity misura lo spazio in "Unità" (metri virtuali). Il bridge tra i pixel disegnati e il codice è il PPU.

### Formula Fondamentale

$$1 \text{ Unità Unity} = 100 \text{ Pixel}$$

### Calcolo della Dimensione

$$\text{Dimensione in Unità} = \frac{\text{Dimensione in Pixel}}{100}$$

### Esempio Pratico
Una racchetta larga 14 pixel e alta 60 pixel:

$$\text{Larghezza} = \frac{14}{100} = 0.14 \text{ unità}$$

$$\text{Altezza} = \frac{60}{100} = 0.60 \text{ unità}$$

### Implicazione
Il Box Collider 2D deve avere Size **0.14 × 0.60** per combaciare perfettamente con la grafica.

---

## 4. Moltiplicazione delle Trasformazioni

### Formula Generale

$$\text{Dimensione Finale} = \text{Scale del Transform} \times \text{Size del Collider}$$

### Esempio
Se lo Scale è 1 e il Collider Size è 0.14:

$$\text{Dimensione Finale} = 1 \times 0.14 = 0.14 \text{ unità}$$

Se aumenti lo Scale a 2:

$$\text{Dimensione Finale} = 2 \times 0.14 = 0.28 \text{ unità}$$

### Attenzione all'Errore Comune
Se porti lo Scale a 2 e poi modifichi manualmente il Collider Size a 0.28, otterrai:

$$\text{Dimensione Finale} = 2 \times 0.28 = 0.56 \text{ unità}$$

Questo raddoppia il valore desiderato.

---

## 5. Architettura degli Oggetti: Parent-Child

### Struttura

```
Paddle (Parent - Oggetto Vuoto)
├─ Script (Logica di movimento)
├─ Rigidbody 2D (Fisica)
├─ Box Collider 2D (Forma fisica)
└─ Sprite (Child - Solo grafica visuale)
```

### Vantaggi
Questa separazione permette di:
- Scalare o ruotare la grafica senza rompere la fisica
- Gestire trasformazioni indipendenti
- Mantenere Z = 0 per evitare conflitti 2D/Rendering

---

## 6. Fisica: Gravità e Movimento

### Impostazioni Rigidbody 2D

| Parametro | Valore | Motivo |
|-----------|--------|--------|
| **Gravity Scale** | 0 | Gli oggetti devono galleggiare |
| **Body Type** | Dynamic | Rispondono alla fisica |
| **Velocity** | Impostata via codice | Movimento costante e stop immediati |

### Movimento: Velocity vs AddForce

**Velocity** (usato in Pong):
```csharp
rb.velocity = new Vector2(0, input * speed);
```
Movimento costante, stop immediato.

**AddForce** (simulazione realistica):
```csharp
rb.AddForce(direction * force);
```
Accumula spinta gradualmente.

---

## 7. Input: Asse Verticale

### Formula di Movimento

$$\text{Movimento Y} = \text{Input Axis} \times \text{Speed}$$

Dove:
- **Input Axis**: Valore tra -1.0 (giù) e 1.0 (su)
- **Speed**: Velocità configurabile dall'Inspector

### Esempio
Se Input.GetAxis("Vertical") = 1.0 e speed = 3:

$$\text{Movimento Y} = 1.0 \times 3 = 3 \text{ unità/secondo}$$

---

## 8. Ordine Visivo: Sorting Layers

L'asse Z non determina l'ordine di disegno. Usiamo Sorting Layers:

| Layer | Order in Layer | Elementi |
|-------|----------------|----------|
| Default | 0 | Sfondo |
| Default | 5 | Paddle e Ball |
| Default | 10 | UI (Punteggio) |

**Regola**: Tutti gli oggetti a Z = 0 per evitare conflitti fisici.

---

## 9. Limiti del Campo (Screen Limits)

### Calcolo della Posizione

Se l'area di gioco è alta 3.6 unità (360 pixel ÷ 100):

$$\text{Altezza totale} = 3.6 \text{ unità}$$

$$\text{Limite Superiore} = \frac{3.6}{2} = 1.8 \text{ unità}$$

$$\text{Limite Inferiore} = -1.8 \text{ unità}$$

Usando un collider a Y = 1.51, si allinea la fisica al bordo visivo dello sfondo (considerando lo spessore grafico).

---

## 10. Rimbalzi: Inversione del Vettore Velocità

### Collisione con Muro (Limit)

Inverte solo l'asse Y:

$$\vec{v}_{\text{dopo}} = (v_x, -v_y)$$

La pallina continua a muoversi orizzontalmente, ma cambia direzione verticale.

### Collisione con Racchetta (Paddle)

Inverte l'asse X e applica il moltiplicatore di difficoltà:

$$\vec{v}_{\text{dopo}} = (-v_x \times m, v_y \times m)$$

Dove **m** = difficultyMultiplier (es. 1.3)

---

## 11. Casualità: Random Direction

### Direzione Iniziale Casuale

Usiamo l'operatore ternario per simulare il lancio di una moneta:

$$\text{Direzione} = \begin{cases} -1 & \text{se } \text{Random.value} > 0.5 \\ 1 & \text{altrimenti} \end{cases}$$

### Velocità Casuale

$$v_x = \text{Random.Range}(\text{min}, \text{max}) \times \text{Direzione}$$

$$v_y = \text{Random.Range}(\text{min}, \text{max}) \times \text{Direzione}$$

---

## 12. Sistema di Scoring

### Rilevamento del Punto

Non usiamo collider laterali. Monitoriamo la coordinata X:

$$\text{Punto Giocatore 1} \quad \text{se} \quad x > 3.4$$

$$\text{Punto Giocatore 2} \quad \text{se} \quad x < -3.4$$

### Memoria: Destroy
Fondamentale eliminare la pallina vecchia prima di crearne una nuova:

```csharp
Destroy(currentBall.gameObject);
```

Evita che palline "fantasma" continuino a occupare RAM.

---

## 13. Difficoltà Crescente: Moltiplicatore Esponenziale

### Formula di Accelerazione

Dopo **n** rimbalzi sulla racchetta:

$$v_{\text{finale}} = v_{\text{iniziale}} \times m^n$$

Dove **m** = difficultyMultiplier

### Esempio Pratico
Con velocità iniziale 1.0 e moltiplicatore 1.3:

| Rimbalzo | Calcolo | Velocità |
|----------|---------|----------|
| 0 | 1.0 | 1.0 |
| 1 | $1.0 \times 1.3$ | 1.3 |
| 2 | $1.3 \times 1.3$ | 1.69 |
| 3 | $1.69 \times 1.3$ | 2.197 |
| 4 | $2.197 \times 1.3$ | 2.856 |
| 5 | $2.856 \times 1.3$ | 3.71 |

**Risultato**: Dopo 5 rimbalzi, la pallina è **3.71 volte più veloce**.

---

## 14. Speed Capping: Prevenire il Tunneling

### Il Problema
Se la velocità diventa troppo alta, la pallina può "saltare" oltre i collider in un singolo frame.

### Formula di Limitazione

$$v_{\text{capped}} = \begin{cases} v & \text{se } |v| \leq v_{\max} \\ \frac{v}{|v|} \times v_{\max} & \text{se } |v| > v_{\max} \end{cases}$$

Dove:
- $|v|$ = magnitudine della velocità (lunghezza del vettore)
- $\frac{v}{|v|}$ = direzione normalizzata (solo direzione, magnitudine = 1)

### Esempio
Se v = (3.0, 2.0) e maxSpeed = 3.0:

$$|v| = \sqrt{3.0^2 + 2.0^2} = \sqrt{13} \approx 3.606$$

Poiché 3.606 > 3.0, capping:

$$v_{\text{capped}} = \frac{(3.0, 2.0)}{3.606} \times 3.0 \approx (2.49, 1.66)$$

---
